In [9]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
from nltools.data import Brain_Data
from nltools.mask import expand_mask, roi_to_brain
from nilearn.plotting import view_img_on_surf, view_img
from nltools.stats import isc, threshold, fdr


def extract_event_segments(event_vector, event_duration=4, min_non_event_duration=4):
    """
    Extract event segments and non-event segments from a binary vector.
    
    Parameters:
    -----------
    event_vector : array-like
        Binary vector of 0s and 1s where 1s indicate events.
    event_duration : int, optional
        How many time points to include after each event onset (default: 5).
    min_non_event_duration : int, optional
        Minimum duration for non-event segments to be included (default: 5).
    
    Returns:
    --------
    tuple
        (event_segments, non_event_segments)
        Each is a list of tuples (start, end) indicating segment boundaries (inclusive).
    """
    event_vector = np.array(event_vector)
    n = len(event_vector)
    
    # Find event onsets (1s that follow 0s or are at the beginning)
    onsets = np.where(np.diff(np.hstack(([0], event_vector))) == 1)[0]
    
    # Initialize event segments list
    event_segments = []
    
    # Process each onset
    for onset in onsets:
        # Calculate the end of this segment (onset + duration - 1)
        # Ensuring it doesn't exceed the vector length
        end = min(onset + event_duration - 1, n - 1)
        
        # Check if we need to merge with the previous segment
        if event_segments and onset <= event_segments[-1][1]:
            # Merge by extending the end of the previous segment if needed
            event_segments[-1] = (event_segments[-1][0], max(event_segments[-1][1], end))
        else:
            # Create a new segment
            event_segments.append((onset, end))
    
    # Create non-event segments by negating event segments
    non_event_segments = []
    
    # If no events, the whole vector is a non-event
    if not event_segments:
        non_event_segments = [(0, n - 1)]
    else:
        # Handle potential non-event segment before first event
        if event_segments[0][0] > 0:
            start = 0
            end = event_segments[0][0] - 1
            if end - start + 1 >= min_non_event_duration:
                non_event_segments.append((start, end))
        
        # Handle non-event segments between events
        for i in range(len(event_segments) - 1):
            start = event_segments[i][1] + 1
            end = event_segments[i + 1][0] - 1
            if end >= start and end - start + 1 >= min_non_event_duration:
                non_event_segments.append((start, end))
        
        # Handle potential non-event segment after last event
        if event_segments[-1][1] < n - 1:
            start = event_segments[-1][1] + 1
            end = n - 1
            if end - start + 1 >= min_non_event_duration:
                non_event_segments.append((start, end))
    
    return event_segments, non_event_segments

In [10]:
folder_path = '/Users/schmaelz/Documents/01_GITHUB/nomcomm/GITHUB_PAPERS_WORKING/friends/data/00_raw_fmri/'  # <-- change to your folder
raw_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith('.npy') and "._" not in file:
            raw_files.append(file[5:12])

raw_files.sort()
raw_files[:3]

['s01e01a', 's01e01b', 's01e02a']

In [27]:
def run_concat_isc(curr_show_name, hrf_shift = 3):

    #curr_show_name = raw_files[0]
    
    laughter_file = '../data/02_laughter/laughter_annotations/friends_' + curr_show_name + '.csv'
    print(laughter_file)
    raw_data_path = '../data/00_raw_fmri/task-' + curr_show_name + '.npy'
    
    df = pd.read_csv(laughter_file)
    laughter_segments, non_laughter_segments = extract_event_segments(df.ls.values)
            
    data = np.load(raw_data_path)
    [x, y, z] = data.shape
                      
    #for the laughter events ISC    
    # concatenate segments
    result_laughter = None  # start with nothing
    print('n laughter segments:')
    print(len(laughter_segments))
    for current_event_index in range(len(laughter_segments)):
        curr_onset   = laughter_segments[current_event_index][0] + hrf_shift
        curr_offset  = laughter_segments[current_event_index][1] + hrf_shift
        if curr_offset > y: #catch if we overflow
            curr_offset = y
        if result_laughter is None:
            result_laughter = data[:, curr_onset:curr_offset, :]
        else:
            # Subsequent arrays: concatenate along axis=1
            result_laughter = np.concatenate((result_laughter, data[:, curr_onset:curr_offset, :]), axis=1)
    
    # loop over rois and computer ISC
    isc_results = []
    for curr_roi in (range(1000)):
        #isc_results = []
        data_roi_df = pd.DataFrame( result_laughter[:, :, curr_roi].T)
        stats_boot = isc(data_roi_df, metric = 'mean', method='phase_randomize', n_samples=1, return_null=False);
        isc_results.append(stats_boot['isc']);
    
    isc_results_laughter = np.asarray(isc_results)
    out_name1 = '../data/02_laughter/laughter_iscs/' + curr_show_name + '_laughter_segment_isc.npy'
    np.save(out_name1, isc_results_laughter)
    
    #for the non laughter events ISC
    # concatenate segments
    result_no_laughter = None  # start with nothing
    print('n nonlaughter segments:')
    print(len(non_laughter_segments))
    for current_event_index in range(len(non_laughter_segments)):
        curr_onset   = non_laughter_segments[current_event_index][0] + hrf_shift
        curr_offset  = non_laughter_segments[current_event_index][1] + hrf_shift
        if curr_offset > y: #catch if we overflow
            curr_offset = y
        if result_no_laughter is None:
            result_no_laughter = data[:, curr_onset:curr_offset, :]
        else:
            # Subsequent arrays: concatenate along axis=1
            result_no_laughter = np.concatenate((result_no_laughter, data[:, curr_onset:curr_offset, :]), axis=1)
    
    # loop over rois and computer ISC
    isc_results = []
    for curr_roi in (range(1000)):
        #isc_results = []
        data_roi_df = pd.DataFrame( result_no_laughter[:, :, curr_roi].T)
        stats_boot = isc(data_roi_df, metric = 'mean', method='phase_randomize', n_samples=1, return_null=False);
        isc_results.append(stats_boot['isc']);
    
    isc_results_no_laughter = np.asarray(isc_results)
    out_name2 = '../data/02_laughter/laughter_iscs/' + curr_show_name + '_no_laughter_segment_isc.npy'
    np.save(out_name2, isc_results_no_laughter)
    
    return isc_results_laughter, isc_results_no_laughter

In [29]:
for curr_show_index in tqdm(range(len(raw_files))):
    curr_show_name = raw_files[curr_show_index]
    run_concat_isc(curr_show_name)
   

  0%|                                                   | 0/280 [00:00<?, ?it/s]

../data/02_laughter/laughter_annotations/friends_s01e01a.csv
n laughter segments:
72
n nonlaughter segments:
30


  0%|▏                                        | 1/280 [00:32<2:29:35, 32.17s/it]

../data/02_laughter/laughter_annotations/friends_s01e01b.csv
n laughter segments:
65
n nonlaughter segments:
27


  1%|▎                                        | 2/280 [01:04<2:29:13, 32.21s/it]

...

 99%|██████████████████████████████████████▋| 278/280 [3:47:49<01:14, 37.38s/it]

../data/02_laughter/laughter_annotations/friends_s06e23a.csv
n laughter segments:
55
n nonlaughter segments:
18


100%|██████████████████████████████████████▊| 279/280 [3:48:27<00:37, 37.43s/it]

../data/02_laughter/laughter_annotations/friends_s06e23b.csv
n laughter segments:
62
n nonlaughter segments:
26


100%|███████████████████████████████████████| 280/280 [3:49:04<00:00, 49.09s/it]
